# Cheating

It's important to know how to cheat. So the first step is to create a system that works exactly like `transformers` API, except it doesn't use `transformers`. We simply write a proxy as follows:

In [1]:
import torch

class FairseqProxy():
    def __init__(self, module):
        self.module = module
        
    @classmethod
    def from_pretrained(cls, mname): 
        return cls(module=torch.hub.load('pytorch/fairseq', mname, checkpoint_file='model1.pt:model2.pt:model3.pt:model4.pt', tokenizer='moses', bpe='fastbpe'))

class FSMTForConditionalGeneration(FairseqProxy):
    
    def generate(self, **tokenized_sentences):
        return self.module.generate(tokenized_sentences['data'])
    
class FSMTTokenizer(FairseqProxy):

    def prepare_seq2seq_batch(self, sentences): # encode
        return {'data': [self.module.encode(sentence) for sentence in sentences]}
    
    def batch_decode(self, batched_hypos):
        return [self.module.decode(hypos[0]['tokens']) for hypos in batched_hypos]

In [4]:
mname = 'transformer.wmt19.ru-en'
tokenizer = FSMTTokenizer.from_pretrained(mname)
model = FSMTForConditionalGeneration.from_pretrained(mname)

Using cache found in /home/stas/.cache/torch/hub/pytorch_fairseq_master
Using cache found in /home/stas/.cache/torch/hub/pytorch_fairseq_master


In [ ]:
batch = tokenizer.prepare_seq2seq_batch(["Машинное обучение - это здорово!"])
output = model.generate(**batch)
decoded = tokenizer.batch_decode(output)[0]
print(decoded)
assert decoded == 'Machine learning is great!'

Success!!!

Practically, unless you want to use some special features from `transformers` this works just fine.